# Load data into Cosmos DB using the MongoDB API

In [36]:
import json
import os
import pandas as pd
import pymongo
import requests
import time
import urllib.parse

from pymongo import UpdateOne, DeleteMany, MongoClient
from models import Job, JobList
from dotenv import load_dotenv
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt

## Establish a connection to the database

In [14]:
load_dotenv()
user = urllib.parse.quote_plus(os.environ.get("user"))
password = urllib.parse.quote_plus(os.environ.get("password"))

In [15]:
DB_URL = os.environ.get("DB_CONNECTION_STRING")
CONNECTION_STRING = f"mongodb+srv://{user}:{password}@{DB_URL}"
client = pymongo.MongoClient(CONNECTION_STRING)
# Create database to hold cosmic works data
# MongoDB will create the database if it does not exist
db = client.cosmic_works

In [16]:
# empty the collections
db.jobs.bulk_write([DeleteMany({})])

BulkWriteResult({'writeErrors': [], 'writeConcernErrors': [], 'nInserted': 0, 'nUpserted': 0, 'nMatched': 0, 'nModified': 0, 'nRemoved': 1, 'upserted': []}, acknowledged=True)

## Load Jobs

In [17]:
jobs_df = pd.read_csv("data/jobs.csv")
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2027 entries, 0 to 2026
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         2027 non-null   object
 1   Role          2027 non-null   object
 2   Company       2027 non-null   object
 3   Created Date  2027 non-null   object
 4   Description   2027 non-null   object
 5   Job Location  2027 non-null   object
dtypes: object(6)
memory usage: 95.1+ KB


In [18]:
from datetime import datetime

# Define a function to convert the DataFrame rows to dictionary format for Pydantic models
def transform_row(row):
    return {
        "roleId": row["Role"],
        "roleName": row["Role"],
        "title": row["Title"],
        "companyName": row["Company"],
        "createdDate": datetime.strptime(row["Created Date"], "%m/%d/%y %H:%M"),
        "description": row["Description"],
        "jobLocation": row["Job Location"]
    }

# Apply the transformation to the DataFrame
jobs_data = jobs_df.apply(transform_row, axis=1).tolist()

In [19]:
# Create Job instances
job_list = JobList(items=[Job(**job) for job in jobs_data])

In [21]:
jobs_collection = db.get_collection('jobs')

# Insert jobs into the collection
for job in job_list.items:
    job_dict = job.dict(by_alias=True)
    if job_dict['_id'] is None:
        del job_dict['_id']  # Remove _id field if it's None
    jobs_collection.insert_one(job_dict)

print("Data uploaded successfully!")

/var/folders/p_/hsf19p9s6_z4zym1nlnln4nr0000gn/T/ipykernel_20562/3589538583.py:5: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  job_dict = job.dict(by_alias=True)


Data uploaded successfully!


## Establish Azure OpenAI connectivity

In [28]:
EMBEDDINGS_DEPLOYMENT_NAME = os.environ.get("EMBEDDINGS_DEPLOYMENT_NAME")
COMPLETIONS_DEPLOYMENT_NAME = os.environ.get("COMPLETIONS_DEPLOYMENT_NAME")
AOAI_ENDPOINT = os.environ.get("AOAI_ENDPOINT")
AOAI_KEY = os.environ.get("AOAI_KEY")
AOAI_API_VERSION = "2023-05-15"

ai_client = AzureOpenAI(
    azure_endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    api_key = AOAI_KEY
    )

## Vectorize and store the embeddings in each document

In [32]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(3))
def generate_embeddings(text: str):
    '''
    Generate embeddings from string of text using the deployed Azure OpenAI API embeddings model.
    This will be used to vectorize document data and incoming user messages for a similarity search with
    the vector index.
    '''
    response = ai_client.embeddings.create(input=text, model=EMBEDDINGS_DEPLOYMENT_NAME)
    embeddings = response.data[0].embedding
    time.sleep(0.5) # rest period to avoid rate limiting on AOAI
    return embeddings

In [33]:
# demonstrate embeddings generation using a test string
test = "hello, world"
print(generate_embeddings(test))

[-0.016903093084692955, -0.006888856645673513, -0.027763785794377327, -0.04649001732468605, -0.011016962118446827, 0.010196549817919731, -0.01406420674175024, -0.004762948025017977, -0.018856454640626907, -0.028388861566781998, 0.02909207157790661, 0.019963359460234642, -0.021812543272972107, -0.006263780873268843, 0.009525896050035954, 0.0065665519796311855, 0.017410967499017715, -0.014350700192153454, 0.011863418854773045, 0.018804365769028664, -0.012495005503296852, -1.7002897948259488e-05, 0.009206846356391907, -0.010281195864081383, -0.009695187211036682, -0.016551487147808075, 0.006986524909734726, -0.01675984635949135, 0.024560272693634033, -0.03815567120909691, 0.0007276273681782186, 0.0034574507735669613, -0.016043614596128464, -0.006322382017970085, 0.01116671971976757, -0.011935042217373848, 0.0009498223080299795, -0.027789831161499023, 0.029534833505749702, -0.011290432885289192, 0.002396124182268977, -0.0071167489513754845, 0.0041606612503528595, -0.013725624419748783, -0.

## Vectorize and update all documents in the Career Path database

In [44]:
def add_collection_content_vector_field(collection_name: str, batch_size: int = 100):
    '''
    Add a new field to the collection to hold the vectorized content of each document.
    '''
    collection = db[collection_name]
    bulk_operations = []
    
    total_docs = collection.count_documents({})
    processed_docs = 0

    while processed_docs < total_docs:
        cursor = collection.find(no_cursor_timeout=True).skip(processed_docs).limit(batch_size)
        batch = list(cursor)
        
        for doc in batch:
            # remove any previous contentVector embeddings
            if "contentVector" in doc:
                del doc["contentVector"]

            # generate embeddings for the document string representation
            content = json.dumps(doc, default=str)
            content_vector = generate_embeddings(content)       

            bulk_operations.append(UpdateOne(
                {"_id": doc["_id"]},
                {"$set": {"contentVector": content_vector}},
                upsert=True
            ))

        # execute bulk operations
        if bulk_operations:
            collection.bulk_write(bulk_operations)
            bulk_operations = []

        processed_docs += len(batch)
        cursor.close()
    
    print("Content vectors added successfully.")

In [45]:
# Add vector field to jobs documents - this will take approximately 3-5 minutes due to rate limiting
add_collection_content_vector_field("jobs")

Content vectors added successfully.


In [46]:
# Create the jobs vector index
db.command({
  'createIndexes': 'jobs',
  'indexes': [
    {
      'name': 'VectorSearchIndex',
      'key': {
        "contentVector": "cosmosSearch"
      },
      'cosmosSearchOptions': {
        'kind': 'vector-ivf',
        'numLists': 1,
        'similarity': 'COS',
        'dimensions': 1536
      }
    }
  ]
})

{'raw': {'defaultShard': {'numIndexesBefore': 1,
   'numIndexesAfter': 2,
   'createdCollectionAutomatically': False,
   'ok': 1}},
 'ok': 1}

## User vector search in vCore-based Azure Cosmos DB for MongoDB

In [57]:
def vector_search(collection_name, query, num_results=3):
    """
    Perform a vector search on the specified collection by vectorizing
    the query and searching the vector index for the most similar documents.

    returns a list of the top num_results most similar documents
    """
    collection = db[collection_name]
    query_embedding = generate_embeddings(query)    
    pipeline = [
        {
            '$search': {
                "cosmosSearch": {
                    "vector": query_embedding,
                    "path": "contentVector",
                    "k": num_results
                },
                "returnStoredSource": True }},
        {'$project': { 'similarityScore': { '$meta': 'searchScore' }, 'document' : '$$ROOT' } }
    ]
    results = collection.aggregate(pipeline)
    return results

def print_job_search_result(result):
    '''
    Print the search result document in a readable format
    '''
    print(f"Similarity Score: {result['similarityScore']}")  
    # print(f"Results: {result['document']}")   

    print(f"Title: {result['document']['title']}")   
    print(f"Role: {result['document']['roleName']}")
    print(f"Company: {result['document']['companyName']}")
    print(f"Location: {result['document']['jobLocation']}")
    print(f"Date Created: {result['document']['createdDate']}")
    print(f"Job Description: {result['document']['description']}")
    print(f"_id: {result['document']['_id']}\n")

In [58]:
query = "What product manager jobs do you have?"
results = vector_search("jobs", query, num_results=4)
for result in results:
    print_job_search_result(result)   

Similarity Score: 0.8310456028269233
Title: Senior Product Manager
Role: Product Management
Company: StrongDM
Location: Remote
Date Created: 2024-05-08 15:08:00
Job Description: StrongDM, a rapidly growing startup, is seeking a Senior Product Manager with a strong security focus. The company values putting people first and aims for customer satisfaction and retention. The ideal candidate will have 7+ years of product management experience in security, a background in enterprise products, familiarity with cloud technologies, strong leadership and communication skills, and a track record of successful product launches and revenue growth. Responsibilities include defining product direction, working cross-functionally, conducting market research, and driving product strategy. The role offers a salary range of $180,000-$210,000 plus equity, with company-sponsored benefits such as medical insurance, 401K, parental leave, and more. This full-time remote position is an opportunity to work in a

In [60]:
query = "What jobs do you have in Boston?"
results = vector_search("jobs", query, num_results=4)
for result in results:
    print_job_search_result(result) 

Similarity Score: 0.7776318523444954
Title: Backend Software Engineer (Boston)
Role: Software Engineering
Company: Lendbuzz
Location: Boston
Date Created: 2024-04-10 15:27:00
Job Description: Lendbuzz, a fintech startup in Boston, is seeking a Backend Software Engineer to join their diverse team. Responsibilities include developing backend infrastructure, collaborating on architecture design, working on event-driven microservices, and contributing to product development. Qualifications include 3+ years of backend development experience, a Bachelor's degree in computer science, proficiency in relational databases, API design, microservices architecture, testing methodologies, and strong interpersonal skills. The company offers a culture that values collaboration, competitive salary, opportunities for growth, great benefits, retirement plan, and flexible vacation policy. The salary for this role ranges from $135,000 to $145,000 annually.
_id: 6643b531854283ab25c40986

Similarity Score: 0

## Use vector serach results in RAG pattern with Chat GPT-3.5

In [62]:
# A system prompt describes the responsibilities, instructions, and persona of the AI.
system_prompt = """
You are a helpful, fun and friendly career advisor for Meander, a tech-focused career management platform. 
Your name is Merlin.
You are designed to answer questions about the job listings on the Meander job board.

Only answer questions related to the information provided in the list of jobs below that are represented
in JSON format.

If you are asked a question that is not in the list, respond with "I don't know."

List of jobs:
"""

In [65]:
def rag_with_vector_search(question: str, num_results: int = 3):
    """
    Use the RAG model to generate a prompt using vector search results based on the
    incoming question.  
    """
    # perform the vector search and build product list
    results = vector_search("jobs", question, num_results=num_results)
    jobs_list = ""
    for result in results:
        if "contentVector" in result["document"]:
            del result["document"]["contentVector"]
        jobs_list += json.dumps(result["document"], indent=4, default=str) + "\n\n"

    # generate prompt for the LLM with vector results
    formatted_prompt = system_prompt + jobs_list

    # prepare the LLM request
    messages = [
        {"role": "system", "content": formatted_prompt},
        {"role": "user", "content": question}
    ]

    completion = ai_client.chat.completions.create(messages=messages, model=COMPLETIONS_DEPLOYMENT_NAME)
    return completion.choices[0].message.content

In [66]:
print(rag_with_vector_search("What types of jobs do you have?", 5))

We have jobs in the following roles:

1. Workplace Strategy
2. People Operations
3. Software Engineering


In [67]:
print(rag_with_vector_search("What are the names of the companies with the most job postings?", 5))

The companies with the most job postings are Iterative Health and Snyk.
